<!-- TODO(ccy): split this notebook into (1) a beginner friendly notebook that
     avoids directly calling into TFX libraries and (2) an advanced notebook
     or notebooks that delve more deeply into each component, along with their
     underlying libraries. -->
# TFX Iterative Development Example
This notebook demonstrates how to use Jupyter notebooks for TFX iterative development.  Here, we walk through the Chicago Taxi example in an interactive Jupyter notebook.

Note: this notebook along with its associated APIs are **experimental** and are
in active development.  Major changes in functionality, behavior and
presentation are expected.

## Setup
First, download data, import modules and set up paths.

### Import packages
We import necessary packages, including standard TFX component classes.

In [1]:
import os
import tempfile
import urllib

import tensorflow as tf
import tfx
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.interactive.interactive_context import InteractiveContext, check_ipython
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import csv_input

/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type


Instructions for updating:
from_feature_spec is a deprecated, use schema_utils.schema_from_feature_spec


### Set up pipeline paths

In [2]:
# Set up paths.

_pipeline_name = 'chicago_taxi_beam'

if check_ipython():
  _tfx_root = tfx.__path__[0]
  _taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')
  # Path which can be listened to by the model server.  Pusher will output the
  # trained model here.
  _serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/taxi_simple')
else:
  # Paths for exported pipeline.
  _tfx_root = os.path.join(os.environ['HOME'], 'tfx')
  _taxi_root = os.path.join(os.environ['HOME'], 'taxi')    
  _serving_model_dir = os.path.join(_taxi_root, 'serving_model')

# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')

_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

### Download example data
We download the sample dataset for use in our TFX pipeline.

In [3]:
# Download the example data.
if check_ipython():
  _data_root = tempfile.mkdtemp(prefix='tfx-data')
  DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
  with open(os.path.join(_data_root, 'data.csv'), 'wb') as f:
      contents = urllib.request.urlopen(DATA_PATH).read()
      f.write(contents)
else:
  _data_root = os.path.join(_taxi_root, 'data', 'simple')

## Create the InteractiveContext
We now create the interactive context.

In [4]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext.
context = InteractiveContext()

## Run TFX components interactively
Next, we construct TFX components and run each one interactively using within the interactive session to obtain `ExecutionResult` objects.

### ExampleGen
`ExampleGen` brings data into the TFX pipeline.

In [5]:
# Use the packaged CSV input data.
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input_base=examples)
context.run(example_gen)

INFO:tensorflow:Run driver for CsvExampleGen


INFO:tensorflow:Run driver for CsvExampleGen


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:14.243518.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:14.243518.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:14.243518 is 1.


...

INFO:tensorflow:Publishing execution id: 1
type_id: 3
properties {
  key: "component_id"
  value {
    string_value: "CsvExampleGen"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "input_config"
  value {
    string_value: "{\n  \"splits\": [\n    {\n      \"name\": \"single_split\",\n      \"pattern\": \"*\"\n    }\n  ]\n}"
  }
}
properties {
  key: "output_config"
  value {
    string_value: "{\n  \"splitConfig\": {\n    \"splits\": [\n      {\n        \"hashBuckets\": 2,\n        \"name\": \"train\"\n      },\n      {\n        \"hashBuckets\": 1,\n        \"name\": \"eval\"\n      }\n    ]\n  }\n}"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:14.243518"
  }

INFO:tensorflow:Publishing execution id: 1
type_id: 3
properties {
  key: "component_id"
  value {
    string_value: "CsvExampleGen"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "input_config"
  value {
    string_value: "{\n  \"splits\": [\n    {\n      \"name\": \"single_split\",\n      \"pattern\": \"*\"\n    }\n  ]\n}"
  }
}
properties {
  key: "output_config"
  value {
    string_value: "{\n  \"splitConfig\": {\n    \"splits\": [\n      {\n        \"hashBuckets\": 2,\n        \"name\": \"train\"\n      },\n      {\n        \"hashBuckets\": 1,\n        \"name\": \"eval\"\n      }\n    ]\n  }\n}"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:14.243518"
  }

INFO:tensorflow:Published execution with final outputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]}


INFO:tensorflow:Published execution with final outputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]}


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]
        ))

### StatisticsGen (using Tensorflow Data Validation)
`StatisticsGen` computes statistics for visualization and example validation. This uses the [Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

#### Run TFDV statistics computation using the StatisticsGen component

In [6]:
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(
    input_data=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:tensorflow:Run driver for StatisticsGen


INFO:tensorflow:Run driver for StatisticsGen


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:25.072102.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:25.072102.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:25.072102 is 2.


...

INFO:tensorflow:Publishing execution id: 2
type_id: 5
properties {
  key: "component_id"
  value {
    string_value: "StatisticsGen"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:25.072102"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'input_data': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]} and outputs {'output': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedr

INFO:tensorflow:Publishing execution id: 2
type_id: 5
properties {
  key: "component_id"
  value {
    string_value: "StatisticsGen"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:25.072102"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'input_data': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]} and outputs {'output': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedr

INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)]}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)]}


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)]
        ))

#### Import TFDV and visualize the statistics result

In [7]:
# Import TFDV and get the train statistics path.
if check_ipython():
  import tensorflow_data_validation as tfdv
  from tfx.types.artifact_utils import get_split_uri
  
  artifact_list = statistics_gen.outputs['output'].get()
  train_artifact_uri = get_split_uri(artifact_list, 'train')
  train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')

In [8]:
# Load statistics and visualize training data stats.
if check_ipython():
  train_stats = tfdv.load_statistics(train_stats_path)
  tfdv.visualize_statistics(train_stats)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### SchemaGen (using Tensorflow Data Validation)
`SchemaGen` generates a schema for your data based on computed statistics. This component also uses the [Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

#### Run TFDV schema inference using the SchemaGen component

In [9]:
# Generates schema based on statistics files.
infer_schema = SchemaGen(
    stats=statistics_gen.outputs['output'],
    infer_feature_shape=False)
context.run(infer_schema)

INFO:tensorflow:Run driver for SchemaGen


INFO:tensorflow:Run driver for SchemaGen


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.400867.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.400867.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.400867 is 3.


...

INFO:tensorflow:Publishing execution id: 3
type_id: 7
properties {
  key: "component_id"
  value {
    string_value: "SchemaGen"
  }
}
properties {
  key: "infer_feature_shape"
  value {
    string_value: "False"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.400867"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'stats': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)]} and outputs {'output': [Artifact(typ

INFO:tensorflow:Publishing execution id: 3
type_id: 7
properties {
  key: "component_id"
  value {
    string_value: "SchemaGen"
  }
}
properties {
  key: "infer_feature_shape"
  value {
    string_value: "False"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.400867"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'stats': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)]} and outputs {'output': [Artifact(typ

INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/SchemaGen/output/3/, split: , id: 6)]}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/SchemaGen/output/3/, split: , id: 6)]}


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/SchemaGen/output/3/, split: , id: 6)]
        ))

#### Visualize the inferred schema

In [10]:
if check_ipython():
  # Get the schema path.
  schema_dir = infer_schema.outputs['output'].get()[0].uri
  schema_path = os.path.join(schema_dir, 'schema.pbtxt')

  # Load and visualize the generated schema.
  schema = tfdv.load_schema_text(schema_path)
  tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,optional,single,'company'
'payment_type',STRING,required,single,'payment_type'
'trip_start_month',INT,required,single,-
'pickup_longitude',FLOAT,required,single,-
'trip_start_hour',INT,required,single,-
'dropoff_longitude',FLOAT,optional,single,-
'dropoff_community_area',FLOAT,optional,single,-
'trip_seconds',FLOAT,optional,single,-
'pickup_community_area',INT,required,single,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2733 - 74600 Benny Jona', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - Ilie Malec', '4053 - 40193 Adwar H. Nikola', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5006 - Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - 88805 Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '6488 - 83287 Zuha Taxi', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Prcard', 'Unknown'"


### ExampleValidator
`ExampleValidator` performs anomaly detection based on computed statistics and your data schema.

#### Run TFDV data validation using the ExampleValidation component

In [11]:
# Performs anomaly detection based on statistics and data schema.
validate_stats = ExampleValidator(
    stats=statistics_gen.outputs['output'],
    schema=infer_schema.outputs['output'])
context.run(validate_stats)

INFO:tensorflow:Run driver for ExampleValidator


INFO:tensorflow:Run driver for ExampleValidator


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.591902.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.591902.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.591902 is 4.


...

INFO:tensorflow:Publishing execution id: 4
type_id: 9
properties {
  key: "component_id"
  value {
    string_value: "ExampleValidator"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.591902"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'stats': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)], 'schema': [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/SchemaGe

INFO:tensorflow:Publishing execution id: 4
type_id: 9
properties {
  key: "component_id"
  value {
    string_value: "ExampleValidator"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.591902"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'stats': [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/train/, split: train, id: 4), Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/StatisticsGen/output/2/eval/, split: eval, id: 5)], 'schema': [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/SchemaGe

INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ExampleValidator/output/4/, split: , id: 7)]}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ExampleValidator/output/4/, split: , id: 7)]}


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ExampleValidator/output/4/, split: , id: 7)]
        ))

#### Visualize the detected anomalies

In [12]:
if check_ipython():
  # Get the validation path.
  validation_dir = validate_stats.outputs['output'].get()[0].uri
  anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

  # Load and visualize the anomalies.
  anomalies = tfdv.load_anomalies_text(anomalies_path)
  tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'company',Unexpected string values,"Examples contain values missing from the schema: 1085 - 72312 N and W Cab Co (<1%), 2192 - Zeymane Corp (<1%), 2809 - 95474 C & D Cab Co Inc. (<1%), 3897 - 57856 Ilie Malec (<1%), 4615 - Tyrone Henderson (<1%), 585 - Valley Cab Co (<1%), 5874 - Sergey Cab Corp. (<1%), 5997 - AW Services Inc. (<1%), 6057 - 24657 Richard Addo (<1%), 6574 - Babylon Express Inc. (<1%), 6743 - Luhak Corp (<1%)."


### Transform
`Transform` performs data transformations and feature engineering which are kept in sync for training and serving.

#### Run the Transform component

In [13]:
# Performs transformations and feature engineering in training and serving.
transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=_taxi_module_file)
context.run(transform)

INFO:tensorflow:Run driver for Transform


INFO:tensorflow:Run driver for Transform


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.767449.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.767449.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:34.767449 is 5.


...

INFO:tensorflow:Publishing execution id: 5
type_id: 11
properties {
  key: "checksum_md5"
  value {
    string_value: "4e52cfff5d4b939df7b60870c459c181"
  }
}
properties {
  key: "component_id"
  value {
    string_value: "Transform"
  }
}
properties {
  key: "module_file"
  value {
    string_value: "/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tfx/examples/chicago_taxi_pipeline/taxi_utils.py"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "preprocessing_fn"
  value {
    string_value: "None"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.767449"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'

INFO:tensorflow:Publishing execution id: 5
type_id: 11
properties {
  key: "checksum_md5"
  value {
    string_value: "4e52cfff5d4b939df7b60870c459c181"
  }
}
properties {
  key: "component_id"
  value {
    string_value: "Transform"
  }
}
properties {
  key: "module_file"
  value {
    string_value: "/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tfx/examples/chicago_taxi_pipeline/taxi_utils.py"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "preprocessing_fn"
  value {
    string_value: "None"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:34.767449"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'

INFO:tensorflow:Published execution with final outputs {'transform_output': [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transform_output/5/, split: , id: 8)], 'transformed_examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/train/, split: train, id: 9), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/eval/, split: eval, id: 10)]}


INFO:tensorflow:Published execution with final outputs {'transform_output': [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transform_output/5/, split: , id: 8)], 'transformed_examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/train/, split: train, id: 9), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/eval/, split: eval, id: 10)]}


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transform_output/5/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
        ))

### Trainer
`Trainer` trains your custom model using TF-Learn.

In [14]:
# Uses user-provided Python function that implements a model using TF-Learn.
trainer = Trainer(
    module_file=_taxi_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

INFO:tensorflow:Run driver for Trainer


INFO:tensorflow:Run driver for Trainer


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:54.357939.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:54.357939.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:46:54.357939 is 6.


...

INFO:tensorflow:Publishing execution id: 6
type_id: 13
properties {
  key: "checksum_md5"
  value {
    string_value: "4e52cfff5d4b939df7b60870c459c181"
  }
}
properties {
  key: "component_id"
  value {
    string_value: "Trainer"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "eval_args"
  value {
    string_value: "{\n  \"numSteps\": 125\n}"
  }
}
properties {
  key: "module_file"
  value {
    string_value: "/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tfx/examples/chicago_taxi_pipeline/taxi_utils.py"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:54.357939"
  }
}


INFO:tensorflow:Publishing execution id: 6
type_id: 13
properties {
  key: "checksum_md5"
  value {
    string_value: "4e52cfff5d4b939df7b60870c459c181"
  }
}
properties {
  key: "component_id"
  value {
    string_value: "Trainer"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "eval_args"
  value {
    string_value: "{\n  \"numSteps\": 125\n}"
  }
}
properties {
  key: "module_file"
  value {
    string_value: "/usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline/local/lib/python3.6/site-packages/tfx/examples/chicago_taxi_pipeline/taxi_utils.py"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:46:54.357939"
  }
}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6/, split: , id: 11)]}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6/, split: , id: 11)]}


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6/, split: , id: 11)]
        ))

### Evaluator (using Tensorflow Model Analysis)
The `Evaluator` computes evaluation statistics over features of your model using [Tensorflow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started). In this section, we run TFMA in our TFX pipeline and then visualize the results to analyze the performance of our model.

#### Run TFMA using the Evaluator component

Here, we first define slicing specs for analyzing our data. Next, we run TFMA using these specs to generate results.

In [15]:
# An empty slice spec means the overall slice, that is, the whole dataset.
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

# Data can be sliced along a feature column
# In this case, data is sliced along feature column trip_start_hour.
FEATURE_COLUMN_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['trip_start_hour'])

# Data can be sliced by crossing feature columns
# In this case, slices are computed for trip_start_day x trip_start_month.
FEATURE_COLUMN_CROSS_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['trip_start_day', 'trip_start_month'])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC,
    FEATURE_COLUMN_CROSS_SPEC,
]

In [16]:
# Use TFMA to compute a evaluation statistics over features of a model.
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))
context.run(model_analyzer)

INFO:tensorflow:Run driver for Evaluator


INFO:tensorflow:Run driver for Evaluator


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:08.773079.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:08.773079.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:08.773079 is 7.


...

INFO:tensorflow:Publishing execution id: 7
type_id: 15
properties {
  key: "component_id"
  value {
    string_value: "Evaluator"
  }
}
properties {
  key: "feature_slicing_spec"
  value {
    string_value: "{\n  \"specs\": [\n    {},\n    {\n      \"columnForSlicing\": [\n        \"trip_start_hour\"\n      ]\n    },\n    {\n      \"columnForSlicing\": [\n        \"trip_start_day\",\n        \"trip_start_month\"\n      ]\n    }\n  ]\n}"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:08.773079"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1

INFO:tensorflow:Publishing execution id: 7
type_id: 15
properties {
  key: "component_id"
  value {
    string_value: "Evaluator"
  }
}
properties {
  key: "feature_slicing_spec"
  value {
    string_value: "{\n  \"specs\": [\n    {},\n    {\n      \"columnForSlicing\": [\n        \"trip_start_hour\"\n      ]\n    },\n    {\n      \"columnForSlicing\": [\n        \"trip_start_day\",\n        \"trip_start_month\"\n      ]\n    }\n  ]\n}"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:08.773079"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1

INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Evaluator/output/7/, split: , id: 12)]}


INFO:tensorflow:Published execution with final outputs {'output': [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Evaluator/output/7/, split: , id: 12)]}


ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Evaluator/output/7/, split: , id: 12)]
        ))

#### Get the TFMA output result path

In [17]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri

#### Import TFMA and load the result

In [18]:
if check_ipython():
  import tensorflow_model_analysis as tfma
  
  tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

#### Visualization: Slicing Metrics

To see the slices, either use the name of the column (by setting slicing_column) or provide a tfma.slicer.SingleSliceSpec (by setting slicing_spec). If neither is provided, an overall visualization will be displayed.

The default visualization is the **slice overview** when the number of slices is small. It shows the value of a metric for each slice, sorted by the another metric. It is also possible to set a threshold to filter out slices with smaller weights.

This view also supports the **metrics histogram** as an alternative visualization. It is also the default view when the number of slices is large. The results will be divided into buckets and the number of slices / total weights / both can be visualized. Slices with small weights can be filtered out by setting the threshold. Further filtering can be applied by dragging the grey band. To reset the range, double click the band. Filtering can be used to remove outliers in the visualization and the metrics table below.

In [19]:
if check_ipython():
  # Show data sliced along feature column trip_start_hour.
  tfma.view.render_slicing_metrics(tfma_result, slicing_column='trip_start_hour')

In [20]:
if check_ipython():
  # Show metrics sliced by 'trip_start_day' x 'trip_start_month'.
  tfma.view.render_slicing_metrics(
      tfma_result,
      slicing_spec=tfma.slicer.SingleSliceSpec(
          columns=['trip_start_day','trip_start_month']))

In [21]:
if check_ipython():
  # Show overall metrics.
  tfma.view.render_slicing_metrics(tfma_result)

### ModelValidator
`ModelValidator` performs validation of your candidate model compared to a baseline.

In [22]:
# Performs quality validation of a candidate model (compared to a baseline).
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])
context.run(model_validator)

INFO:tensorflow:Run driver for ModelValidator


INFO:tensorflow:Run driver for ModelValidator


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:15.976440.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:15.976440.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:15.976440 is 8.


...

INFO:tensorflow:Publishing execution id: 8
type_id: 17
properties {
  key: "component_id"
  value {
    string_value: "ModelValidator"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:15.976440"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)], 'model': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6

INFO:tensorflow:Publishing execution id: 8
type_id: 17
properties {
  key: "component_id"
  value {
    string_value: "ModelValidator"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:15.976440"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'examples': [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/train/, split: train, id: 2), Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/CsvExampleGen/examples/1/eval/, split: eval, id: 3)], 'model': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6

INFO:tensorflow:Published execution with final outputs {'blessing': [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ModelValidator/blessing/8/, split: , id: 13)]}


INFO:tensorflow:Published execution with final outputs {'blessing': [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ModelValidator/blessing/8/, split: , id: 13)]}


ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/ModelValidator/blessing/8/, split: , id: 13)]
        ))

### Pusher
`Pusher` checks whether a model has passed validation, and if so, pushes the model to a file destination.

In [23]:
# Checks whether the model passed the validation steps and pushes the model
# to a file destination if check passed.
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:tensorflow:Run driver for Pusher


INFO:tensorflow:Run driver for Pusher


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:21.651273.


INFO:tensorflow:Created run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:21.651273.


INFO:tensorflow:ID of run context interactive-2019-09-13T16_46_14.222336.2019-09-13T16:47:21.651273 is 9.


...

INFO:tensorflow:Publishing execution id: 9
type_id: 19
properties {
  key: "component_id"
  value {
    string_value: "Pusher"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "push_destination"
  value {
    string_value: "{\n  \"filesystem\": {\n    \"baseDirectory\": \"/tmp/tmp7thxyl6i/serving_model/taxi_simple\"\n  }\n}"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:21.651273"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'model_export': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6/, split: , id: 11)], 'model_blessing': [Artifact(type_name: 

INFO:tensorflow:Publishing execution id: 9
type_id: 19
properties {
  key: "component_id"
  value {
    string_value: "Pusher"
  }
}
properties {
  key: "custom_config"
  value {
    string_value: "None"
  }
}
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2019-09-13T16_46_14.222336"
  }
}
properties {
  key: "pipeline_root"
  value {
    string_value: "/tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a"
  }
}
properties {
  key: "push_destination"
  value {
    string_value: "{\n  \"filesystem\": {\n    \"baseDirectory\": \"/tmp/tmp7thxyl6i/serving_model/taxi_simple\"\n  }\n}"
  }
}
properties {
  key: "run_id"
  value {
    string_value: "2019-09-13T16:47:21.651273"
  }
}
properties {
  key: "state"
  value {
    string_value: "complete"
  }
}
, with inputs {'model_export': [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Trainer/output/6/, split: , id: 11)], 'model_blessing': [Artifact(type_name: 

INFO:tensorflow:Published execution with final outputs {'model_push': [Artifact(type_name: ModelPushPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Pusher/model_push/9/, split: , id: 14)]}


INFO:tensorflow:Published execution with final outputs {'model_push': [Artifact(type_name: ModelPushPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Pusher/model_push/9/, split: , id: 14)]}


ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        model_push: Channel(
            type_name: ModelPushPath
            artifacts: [Artifact(type_name: ModelPushPath, uri: /tmp/tfx-interactive-2019-09-13T16_46_14.222336-5zedre0a/Pusher/model_push/9/, split: , id: 14)]
        ))

In [24]:
# Beam pipeline export template.

from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner


components = [
    example_gen, statistics_gen, infer_schema, validate_stats, transform,
    trainer, model_analyzer, model_validator, pusher
]

tf.logging.set_verbosity(tf.logging.INFO)

beam_pipeline = pipeline.Pipeline(
    pipeline_name=_pipeline_name,
    pipeline_root=_pipeline_root,
    components=components,
    enable_cache=True,
    metadata_connection_config=metadata.sqlite_metadata_connection_config(_metadata_path),
    additional_pipeline_args={})

if not check_ipython():
  BeamDagRunner().run(beam_pipeline)

context.export_to_pipeline(notebook_filename='taxi_pipeline_interactive.ipynb',
                           pipeline_filename=None)

Caller filepath: <ipython-input-24-722c1df61a14>
Exporting contents of %s to %s. /usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline_interactive.ipynb /usr/local/google/home/joeyounglee/git-repos/tfx_oss.git/tfx/examples/chicago_taxi_pipeline/taxi_pipeline_interactive_export.py
